# Bokeh plots

This notebook shows how to draw plots of datasets using Bokeh as the backend.

In [ ]:
import emsarray
import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap


output_notebook()


def wgs84_to_web_mercator(data, lon="lons", lat="lats"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    data["xs"] = [r * (k * np.pi/180.0) for r in data["lons"]]
    data["ys"] = [np.log(np.tan((90 + r) * np.pi/360.0)) * k for r in data["lats"]]
    return data

In [ ]:
ds = emsarray.tutorial.open_dataset("fraser")
ds

In [ ]:
temp = ds.ems.ravel(ds['temp'].isel(time=1, k=-1))[ds.ems.mask]

# Convert the dataset polygons and the scalar values in to a structure that bokeh can use
polygons = ds.ems.polygons[ds.ems.mask]
polygon_coords = [np.asarray(p.exterior.coords) for p in polygons]
data = wgs84_to_web_mercator({
    'lons': [pc[:, 0] for pc in polygon_coords],
    'lats': [pc[:, 1] for pc in polygon_coords],
    'temp': temp.values,
})

# Make a Bokeh figure
fig = figure(x_axis_type="mercator", y_axis_type="mercator", frame_width=800, frame_height=600)
fig.add_tile("CartoDB Positron")

# Add the dataset polygons
colour = LinearColorMapper(Viridis256, low=np.nanmin(temp.values), high=np.nanmax(temp.values), nan_color=(0, 0, 0, 0))
fig.patches(
    'xs', 'ys', source=data,
    fill_color={'field': 'temp', 'transform': colour},
    line_color={'field': 'temp', 'transform': colour})

# Add a colour bar
cb = ColorBar(color_mapper=colour, title="Sea surface temperature")
fig.add_layout(cb, 'right')

# Done!
show(fig)